In [17]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,models
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Dropout,Flatten,MaxPool2D,Rescaling,Input
import matplotlib.pyplot as plt

In [ ]:
data_train= keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Fruits_Vegetables/train',
    labels='inferred',
    batch_size=32,
    image_size=(180, 180),
    shuffle=True,
)

data_val= keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Fruits_Vegetables/validation',
    labels='inferred',

    batch_size=32,
    image_size=(180,180),

)
data_test= keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Fruits_Vegetables/test',
    labels='inferred',

    batch_size=32,
    image_size=(180,180),

)
data_cat=len(data_train.class_names)


Found 3115 files belonging to 36 classes.


<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
import numpy as np
plt.figure(figsize=(10, 10))
for images, labels in data_train.take(1): # take(1) gets one batch
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        # Convert tensors to NumPy array and cast to uint8 for display
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
    plt.show()


Found 351 files belonging to 36 classes.


In [ ]:
model= Sequential()
model.add(Input(shape=(180,180,3)))
model.add(Rescaling(1./255))
#model.add(data_augmentation)
model.add(
    Conv2D(
        filters=16,activation='relu',input_shape=(180,180,3),kernel_size=(3,3)
    )
)

model.add(
    MaxPool2D
    (
    pool_size=(2, 2),
    strides=None,
    padding='valid',
    )
)
model.add(Conv2D
          (filters=64,activation='relu',kernel_size=(3,3)
                )
         )
model.add(
    MaxPool2D
    (
    pool_size=(2, 2),
    strides=None,
    padding='valid',
    )
)
model.add(
    Conv2D
    (
        filters=128,activation='relu',kernel_size=(3,3)
    )
)
model.add(
    MaxPool2D
    (
    pool_size=(2, 2),
    strides=2,
    padding='valid',
    )
)

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dense(data_cat,activation='softmax'))
model.summary()


Found 359 files belonging to 36 classes.


In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 126, 126, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 63, 63, 64)       0         
 2D)                                                  

In [ ]:
history=model.fit(data_train,validation_data=data_val,epochs=30,batch_size=32)

In [ ]:
model.predict(data_test)

In [ ]:
model.evaluate(data_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_loss'],label='val_loss')
plt.plot(history.history['loss'],label='train_loss')
plt.legend()
plt.show()

plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.plot(history.history['accuracy'],label='accuracy')
plt.legend()
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

image_path = '/content/drive/MyDrive/Colab Notebooks/Fruits_Vegetables/test/cabbage/Image_1.jpg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Converting BGR to RGB

image = tf.keras.preprocessing.image.load_img(image_path,target_size=(180,180))
input_arr = tf.keras.preprocessing.image.img_to_array(image)

input_arr = np.array([input_arr])  # Convert single image to a batch.

predictions = model.predict(input_arr)
result_index = np.argmax(predictions) #Return index of max element
print(result_index)
plt.imshow(img)
plt.title('Test Image')
plt.show()

print('Answer:',data_test.class_names[result_index])